<a href="https://colab.research.google.com/github/MaggieAppleton/Colab-Notebooks/blob/main/Claude_Tool_Use_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Claude Tool Use Tutorial

In [ ]:
# Setup packages

%pip install anthropic wikipedia

import anthropic
import wikipedia
import json
from google.colab import userdata
from IPython.display import Markdown, display
import re

api_key = userdata.get('ANTHROPIC_API_KEY')
claude_client = anthropic.Anthropic(api_key=api_key)
model = "claude-3-5-haiku-latest"

print("👍 Claude and wikipedia setup")

👍 Claude and wikipedia setup


# Canonical Tool Use Pattern

How to loop through text and tool responses from Claude

In [ ]:
def math_solver(word_problem):

    system_prompt = """
    You are a mathematics assistant. Help the user solve their math problem by breaking it down and reasoning through it step-by-step.

    Guidelines:
    - Show your work clearly at each step
    - Use the available tools when you need to perform calculations
    - Wrap your reasoning in <reasoning> tags
    - When you have the final answer, wrap it in <final_answer> tags
    - Be precise and show intermediate results
    """

    messages = [{"role": "user", "content": word_problem}]

    while True:
        response = claude_client.messages.create(
            model=model,
            system=system_prompt,
            max_tokens=max_tokens,
            messages=messages,
            tools=tools
        )

        # Add Claude's entire response to conversation history
        messages.append({"role": "assistant", "content": response.content})

        # Process each content block in the response
        has_final_answer = False
        has_tool_use = False

        for content_block in response.content:
            if content_block.type == "text":
                # Handle text content
                text_content = content_block.text
                print(f"Claude says: {text_content}")

                # Check for final answer
                if "<final_answer>" in text_content:
                    has_final_answer = True

            elif content_block.type == "tool_use":
                # Handle tool use
                has_tool_use = True
                tool_name = content_block.name
                tool_input = content_block.input
                tool_id = content_block.id

                print(f"Claude wants to use tool: {tool_name}")

                # Execute the tool
                tool_result = execute_tool(tool_name, tool_input)

                # Add tool result to conversation
                messages.append({
                    "role": "user",
                    "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_id,
                        "content": str(tool_result)
                    }]
                })

        # Check if we have a final answer
        if has_final_answer:
            # Extract final answer from the response
            for content_block in response.content:
                if content_block.type == "text" and "<final_answer>" in content_block.text:
                    answer = extract_answer(content_block.text)
                    return answer

        # If there were tool uses, continue the loop to get Claude's response to the tool results
        if has_tool_use:
            continue

        # If no final answer and no tool use, ask Claude to continue
        messages.append({"role": "user", "content": "Continue to the next step"})

def execute_tool(tool_name, tool_input):
    """Execute a tool and return the result"""
    if tool_name == "calculator":
        # Example: simple calculator tool
        expression = tool_input.get("expression", "")
        try:
            result = eval(expression)  # Note: eval is dangerous, use a proper math parser
            return f"Result: {result}"
        except:
            return "Error: Invalid expression"

    # Add other tools as needed
    return "Tool not implemented"



---



# [Lesson 1](https://github.com/anthropics/courses/blob/master/tool_use/02_your_first_simple_tool.ipynb)

Build a basic tool

All Claude tools need a name, a description, and a JSON input schema

In [ ]:
# Example tool definition for search

search = {
  "name": "search_product",
  "description": "Search for a product by name or keyword and return its current price and availability.",
  "input_schema": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "The product name or search keyword, e.g. 'iPhone 13 Pro' or 'wireless headphones'"
      },
      "category": {
        "type": "string",
        "enum": ["electronics", "clothing", "home", "toys", "sports"],
        "description": "The product category to narrow down the search results"
      },
      "max_price": {
        "type": "number",
        "description": "The maximum price of the product, used to filter the search results"
      }
    },
    "required": ["query"]
  }
}

In [ ]:
# Practice writing a tool definition

def inventory_lookup(product_name, max_results):
  return "This function looks stuff up"

inventory_lookup = {
    "name": "inventory lookup",
    "description": "Look up what products we have in our store inventory",
    "input_schema": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The name of the product we want to look up"
            },
            "max_results": {
                "type": "number",
                "description": "The maximum number of results to return"
            }
        },
        "required": ["product_name", "max_results"]
    }
}

# Giving Claude a calculator

In [ ]:
# Setup our calculator function

def calculator(operation, num1, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 == 0:
            raise ValueError("Cannot divide by zero.")
        return num1 / num2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

display(calculator("add", 10, 3))

display(calculator("divide", 200, 25))


13

8.0

In [ ]:
# Define our calculator tool

calculator_tool = {
    "name": "calculator",
    "description": "A simple calculator that performs basic arithmetic operations.",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform."
            },
            "num1": {
                "type": "number",
                "description": "The first operand."
            },
            "num2": {
                "type": "number",
                "description": "The second operand."
            }
        },
        "required": ["operation", "operand1", "operand2"]
    }
}

In [ ]:
# Ask Claude a calculation question

response = claude_client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content": "Multiply 1984135 by 9343116. Only respond with the result"}],
    system="You have access to tools, but only use them when necessary.  If a tool is not required, respond as normal",
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_tool]
)

display(response)

print(f"\nStop reason: {response.stop_reason}")
print(f"Tool to call: {response.content[0].name}")
print(f"Inputs: {response.content[0].input}")

Message(id='msg_01VcT1AVWxUk4oFrFa6wtRZp', content=[ToolUseBlock(id='toolu_013q5sqPn5JEiKCVSAu5u7gU', input={'num1': 1984135, 'num2': 9343116, 'operation': 'multiply'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=443, output_tokens=91, server_tool_use=None, service_tier='standard'))


Stop reason: tool_use
Tool to call: calculator
Inputs: {'num1': 1984135, 'num2': 9343116, 'operation': 'multiply'}


In [ ]:
# Use Claude's response to run the calculation

operation = response.content[0].input["operation"]
num1 = response.content[0].input["num1"]
num2 = response.content[0].input["num2"]

result = calculator(operation, num1, num2)

print(result)

18538003464660


In [ ]:
# Create a general Claude call that can use calculator if needed

def prompt_claude(prompt):
  messages = [{"role": "user", "content": prompt}]
  response = claude_client.messages.create(
      model="claude-3-haiku-20240307",
      # Tell Claude to only use tools when necessary.
      system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
      messages=messages,
      max_tokens=200,
      tools=[calculator_tool],
  )

  display(response)

  # If the reason Claude stopped was to use a tool, get the tool name and input
  if response.stop_reason == "tool_use":
    # Access the last item in the list – in this case, the ToolUseBlock. TextBlock is the first item.
    tool_block = response.content[-1]
    tool_name = tool_block.name
    tool_input = tool_block.input

    # If the tool is the calculator, try using the calculator (and catch the error)
    if tool_name == "calculator":
      try:
        result = calculator(tool_input["operation"], tool_input["num1"], tool_input["num2"])
        print("\nClaude says:", response.content[0].text)
        print("\nCalculator result is:", result)
      except ValueError as e:
        print(f"Error: {str(e)}")

  elif response.stop_reason == "end_turn":
    print("\nClaude chose not to use a tool.\nResponse:", response.content[0].text)

In [ ]:
prompt_claude("I have 43 bananas to share between 4 toddlers. How many bananas does each toddler get?")

Message(id='msg_011oP8AiZjoxMMwcLHXNB2nT', content=[TextBlock(citations=None, text="Okay, let's calculate how many bananas each toddler gets.", type='text'), ToolUseBlock(id='toolu_01EHg3WZupuuVg49UK3ukBqZ', input={'num1': 43, 'num2': 4, 'operation': 'divide'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=452, output_tokens=104, server_tool_use=None, service_tier='standard'))


Claude says: Okay, let's calculate how many bananas each toddler gets.

Calculator result is: 10.75


In [ ]:
prompt_claude("Where is Paris?")

Message(id='msg_01Fvk5tQbS3HSRZhqABqA8CX', content=[TextBlock(citations=None, text="Paris is the capital and largest city of France. It is located in northern central France, along the Seine River, in the Île-de-France region.\n\nSome key facts about the location of Paris:\n\n- It is situated on the northern central part of the French national territory, roughly 150 kilometers (93 miles) from the English Channel coast.\n\n- Paris is located in the north-central part of the country, about 280 kilometers (175 miles) southeast of the English Channel, 450 kilometers (280 miles) south of the North Sea, and 600 kilometers (375 miles) east of the Atlantic Ocean.\n\n- The city is centered on two small islands (Île de la Cité and Île Saint-Louis) in the middle of the Seine river, which divides the city into the Left Bank (on the south) and the Right Bank (on the north).\n\n- The historic center of Paris is on the Île de la Cité. Many of the city's landmarks and monuments, including Notre-Dame C


Claude chose not to use a tool.
Response: Paris is the capital and largest city of France. It is located in northern central France, along the Seine River, in the Île-de-France region.

Some key facts about the location of Paris:

- It is situated on the northern central part of the French national territory, roughly 150 kilometers (93 miles) from the English Channel coast.

- Paris is located in the north-central part of the country, about 280 kilometers (175 miles) southeast of the English Channel, 450 kilometers (280 miles) south of the North Sea, and 600 kilometers (375 miles) east of the Atlantic Ocean.

- The city is centered on two small islands (Île de la Cité and Île Saint-Louis) in the middle of the Seine river, which divides the city into the Left Bank (on the south) and the Right Bank (on the north).

- The historic center of Paris is on the Île de la Cité. Many of the city's landmarks and monuments, including Notre-Dame Cathedral, are located on or near this island.

So i

# Exercise: Make a Research Assistant

In [ ]:
# Setup wikipedia tools

import wikipedia

def wikipedia_lookup(topic, article_titles):
  """Lookup real articles on Wikipedia"""
  wikipedia_articles = []
  for titles in article_titles:
    results = wikipedia.search(titles)
    try:
      page = wikipedia.page(results[0])
      title = page.title
      url = page.url
      wikipedia_articles.append({"title": title, "url": url})
    except:
      continue

  add_to_reading_list(wikipedia_articles, topic)
  print(wikipedia_articles)

def add_to_reading_list(articles, topic):
  """Add real wikipedia articles to markdown file"""
  with open("/content/research_reading.md", "a", encoding="utf-8") as file:
    file.write(f"## {topic} \n")
    for article in articles:
      title = article["title"]
      url = article["url"]
      file.write(f"* [{title}]({url}) \n")
    file.write(f"\n\n")

In [ ]:
# Define wikipedia lookup tool

wikipedia_lookup_tool = {
    "name": "wikipedia_lookup",
    "description": "Lookup the titles and URLs of real Wikipedia articles",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The research topic we need to find relevant wikipedia articles for"
            },
            "article_titles": {
                "type": "array",
                "description": "A list of article titles that we need to find real matching wikipedia articles for"
            }
        },
        "required": ["topic", "article_titles"]
    }
}

In [ ]:
# Get Claude to generate article titles

def get_research_help(research_topic, num_articles=10):
  """Get Claude to generate wikipedia article titles, then check them against wikipedia"""

  messages = [{"role": "user", "content": f"My research topic is: {research_topic}"}]

  response = claude_client.messages.create(
      messages=messages,
      model="claude-sonnet-4-20250514",
      max_tokens=300,
      system=f"You are a helpful research assistant who specialises in suggesting relevant wikipedia articles. Return a list of {num_articles} wikipedia article titles related to the user's research topic in this format: ['Greek Orthodox Church', 'Greek Christianity', 'Church of Cyprus']. Once you have made a list of suggested articles, use your 'wikipedia_lookup' tool to get the real titles and URLs of these articles.",
      tools=[wikipedia_lookup_tool]
  )

  # display(response)

  # print(f"\nStop reason: {response.stop_reason}")
  # print(f"\nClaude says: {response.content[0].text}")
  # print(f"Tool to call: {response.content[-1].name}")
  # print(f"Inputs: {response.content[-1].input}")

  stop_reason = response.stop_reason

  if stop_reason == "tool_use":
    tool_block = response.content[-1]
    tool_name = tool_block.name

    if tool_name == "wikipedia_lookup":
      try:
        titles = tool_block.input["article_titles"]
        topic = tool_block.input["topic"]

        print(f"Claude says: {response.content[0].text}")

        results = wikipedia_lookup(topic, titles)
      except Exception as e:
        print(f"Error: {e}")
  elif stop_reason == "end_turn":
    print(f"Claude says: {response.content[0].text}")

In [ ]:
get_research_help("Sourdough bread", 5)

Message(id='msg_01HGcYx2TwRE8ESft8p3oDfp', content=[TextBlock(citations=None, text='Okay, let me suggest some relevant Wikipedia articles for your research topic of sourdough bread:', type='text'), ToolUseBlock(id='toolu_01JKrDdMbWTWdac5jo5ggs4k', input={'topic': 'Sourdough bread', 'article_titles': ['Sourdough', 'Bread', 'Fermentation (food)', 'Baking', 'Yeast']}, name='wikipedia_lookup', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=476, output_tokens=123, server_tool_use=None, service_tier='standard'))

Claude says: Okay, let me suggest some relevant Wikipedia articles for your research topic of sourdough bread:
[{'title': 'Sourdough', 'url': 'https://en.wikipedia.org/wiki/Sourdough'}, {'title': 'Breast', 'url': 'https://en.wikipedia.org/wiki/Breast'}, {'title': 'Fermentation in food processing', 'url': 'https://en.wikipedia.org/wiki/Fermentation_in_food_processing'}, {'title': 'Bank', 'url': 'https://en.wikipedia.org/wiki/Bank'}, {'title': 'East', 'url': 'https://en.wikipedia.org/wiki/East'}]


# [Lesson 2](https://github.com/anthropics/courses/blob/master/tool_use/03_structured_outputs.ipynb)

All of Claude's tool responses are JSON by default. We can use this when we want an answer structured in JSON.

We can get Claude to return "inputs" for a tool but then not use them in the tool.

We can force Claude to use a specific tool to get the output in JSON using `tool_choice`

In [ ]:
import json

tools = [
    {
        "name": "print_sentiment",
        "description": "Prints the sentiment scores of a given text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
                "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
                "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
        }
    }
]

def analyse_sentiment(content):

  messages=f"""
  <text>{content}</text>

  Analyse the sentiment of this text and return the scores using the print_sentiment tool.
  """

  response = claude_client.messages.create(
      model="claude-3-haiku-20240307",
      max_tokens=200,
      tools=tools,
      tool_choice={"type": "tool", "name": "print_sentiment"}, # Make Claude use this tool
      messages=[{"role": "user", "content": messages}]
  )

  json_sentiment = None
  for content in response.content:
    if content.type == "tool_use" and content.name == "print_sentiment":
      json_sentiment = content.input
      break

  if json_sentiment:
    print("Sentiment analysis:")
    print(json.dumps(json_sentiment, indent=2))
  else:
    print("No sentiment analysis found in response")

In [ ]:
analyse_sentiment("Strawberries are crap")

Sentiment analysis:
{
  "positive_score": 0.0,
  "negative_score": 0.5,
  "neutral_score": 0.5
}


We can use this technique to extract structured information from freeform text.

In [ ]:
tools = [
    {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "entities": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The extracted entity name."},
                            "type": {"type": "string", "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION)."},
                            "context": {"type": "string", "description": "The context in which the entity appears in the text."}
                        },
                        "required": ["name", "type", "context"]
                    }
                }
            },
            "required": ["entities"]
        }
    }
]

text = "Maggie is applying to Anthropic. Julian and Joel are Maggie's contacts there. She lives in London with Dave. Dave works at nPlan."

query = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

response = claude_client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1000,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_entities"},
    messages=[{"role": "user", "content": query}]
)

json_entities = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_entities":
        json_entities = content.input
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{
  "entities": [
    {
      "name": "Maggie",
      "type": "PERSON",
      "context": "Maggie is applying to Anthropic"
    },
    {
      "name": "Anthropic",
      "type": "ORGANIZATION",
      "context": "Maggie is applying to Anthropic"
    },
    {
      "name": "Julian",
      "type": "PERSON",
      "context": "Julian and Joel are Maggie's contacts there"
    },
    {
      "name": "Joel",
      "type": "PERSON",
      "context": "Julian and Joel are Maggie's contacts there"
    },
    {
      "name": "London",
      "type": "LOCATION",
      "context": "She lives in London with Dave"
    },
    {
      "name": "Dave",
      "type": "PERSON",
      "context": "She lives in London with Dave. Dave works at nPlan"
    },
    {
      "name": "nPlan",
      "type": "ORGANIZATION",
      "context": "Dave works at nPlan"
    }
  ]
}


# Translation Exercise

Making a translator tool that returns structured JSON

In [ ]:
tools=[
    {
        "name": "print_translation", # Note that we make Claude think we're printing the translation, not having the tool do the translation
        "description": "Print the translation of a word or phrase from English into Spanish, French, Japanese, and Arabic",
        "input_schema": {
            "type": "object",
            "properties": {
                "english": {
                    "type": "string",
                    "description": "The original phrase in english"
                },
                "spanish": {
                    "type": "string",
                    "description": "The phrase translated into spanish"
                },
                "french": {
                    "type": "string",
                    "description": "The phrase translated into french"
                },
                "japanese": {
                    "type": "string",
                    "description": "The phrase translated into japanese"
                },
                "arabic": {
                    "type": "string",
                    "description": "The phrase translated into arabic"
                },
            },
            "required": ["english", "spanish", "french", "japanese", "arabic"]
        }
    }
]

def translate(phrase):

  # Setup query
  query = f"""
  <text>
  {phrase}
  </text>

  Use the print_translation tool
  """

  # Pass to Claude
  response = claude_client.messages.create(
      model="claude-sonnet-4-20250514",
      max_tokens=500,
      tools=tools,
      messages=[{"role": "user", "content": query}]
  )

  # Process the results
  translation_json = None
  for content in response.content:
    if content.type == "tool_use" and content.name == "print_translation":
      translation_json = content.input
      break

  if translation_json:
    print("Translation:")
    print(json.dumps(translation_json, ensure_ascii=False, indent=2))
  else:
    print("No translation available")

In [ ]:
translate("Where is my bunny?")

Translation:
{
  "english": "Where is my bunny?",
  "spanish": "¿Dónde está mi conejito?",
  "french": "Où est mon lapin?",
  "japanese": "私のうさぎはどこですか？",
  "arabic": "أين أرنبي؟"
}


# [Lesson 3](https://github.com/anthropics/courses/blob/master/tool_use/04_complete_workflow.ipynb)

Putting together a complete tool use workflow

In [ ]:
# Setup article search tool
article_search_tool = {
    "name": "get_article",
    "description": "Retrieve an up to date Wikipedia article",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "A search term to find a wikipedia article by title"
            }
        },
        "required": ["search_term"]
    }
}

# Get articles from wikipedia
def get_article(search_term):
  results = wikipedia.search(search_term)
  first_result = results[0]
  page = wikipedia.page(first_result, auto_suggest=False)
  return page.content

article = get_article("Superman")
display(article[:500])

'Superman is a superhero created by writer Jerry Siegel and artist Joe Shuster, which first appeared in the comic book Action Comics #1, published in the United States on April 18, 1938. Superman has been regularly published in American comic books since 1938, and has been adapted to other media including radio serials, novels, films, television shows, theater, and video games.\nSuperman was born Kal-El, on the fictional planet Krypton. As a baby, his parents Jor-El and Lara sent him to Earth in a'

In [ ]:
# Complete Claude function to answer the question

def answer_question(question):

  system_prompt = """
    You will be asked a question by the user.
    Answer the question in under 100 words.
    If answering the question requires data you were not trained on, you can use the get_article tool to get the contents of a recent wikipedia article about the topic.
    If you can answer the question without needing to get more information, please do so.
    Only call the tool when needed.
  """

  messages = [{"role": "user", "content": f"<question>{question}</question>"}]

  response = claude_client.messages.create(
      model="claude-3-5-haiku-latest",
      system=system_prompt,
      messages=messages,
      max_tokens=500,
      tools=[article_search_tool]
  )

  if(response.stop_reason == "tool_use"):

      tool_block = next((block for block in response.content if block.type == "tool_use"), None)

      tool_name = tool_block.name
      tool_input = tool_block.input

      messages.append({"role": "assistant", "content": response.content})

      if tool_name == "get_article":
          search_term = tool_input["search_term"]
          print(f"Claude wants to get an article for {search_term}")
          wiki_result = get_article(search_term)

          tool_response = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_block.id,
                    "content": wiki_result
                }
            ]
          }

          messages.append(tool_response)

          response = claude_client.messages.create(
              model="claude-3-5-haiku-latest",
              system=system_prompt,
              messages=messages,
              max_tokens=500,
              tools=[article_search_tool]
          )

          display(Markdown(f"**Claude's final answer**:\n\n {response.content[0].text}"))

  else:
      print("Claude did not call our tool")
      display(Markdown(response.content[0].text))

In [ ]:
answer_question("Who created the Power Puff Girls?")

Claude wants to get an article for Power Puff Girls creator


**Claude's final answer**:

 Based on the Wikipedia article, Craig McCracken created the Power Puff Girls. He originally developed the characters while studying at CalArts, initially creating a short film called "Whoopass Stew!" in 1992. The show was later picked up by Cartoon Network, renamed to The Powerpuff Girls, and premiered on November 18, 1998.

In [ ]:
answer_question("How did Britain come to be so wealthy by the end of the 1900s?")

Claude wants to get an article for British Empire industrial revolution economic power


**Claude's final answer**:

 Britain became wealthy through several key factors: the Industrial Revolution, colonial expansion, technological innovations, and global trade dominance. By leveraging advanced manufacturing, establishing a vast empire, and becoming the world's leading financial center, Britain emerged as the wealthiest nation by the late 1800s. Its naval supremacy and industrial capabilities allowed unprecedented economic growth and global economic control.

In [ ]:
answer_question("Who wrote the book 'Life of Pi'?")

Claude did not call our tool


Yann Martel wrote the book 'Life of Pi', which was published in 2001. It's a philosophical novel that tells the story of a young Indian boy named Pi Patel who survives a shipwreck and finds himself stranded on a lifeboat with a Bengal tiger. The book won the Man Booker Prize in 2002 and was later adapted into a critically acclaimed film directed by Ang Lee in 2012.

In [ ]:
def complex_answer_question(question):
    # Keep searching wikipedia to answer more complex questions

    system_prompt = """
      You will be asked a question by the user.
      Answer the question in under 100 words using the get_article tool to get the contents of a recent wikipedia article about the topic.
      If you don't have enough information to comprehensively answer the question, search again to find other relevant articles on Wikipedia.

      When you're done searching, put your final answer inside a set of <answer> tags.
    """

    messages = [{"role": "user", "content": f"Answer the following question <question>{question}</question>"}]

    while True:
        response = claude_client.messages.create(
            model="claude-3-5-haiku-latest",
            system=system_prompt,
            messages=messages,
            max_tokens=500,
            tools=[article_search_tool]
        )

        if response.stop_reason == "tool_use":
            tool_block = next((block for block in response.content if block.type == "tool_use"), None)

            if tool_block and tool_block.name == "get_article":
                search_term = tool_block.input["search_term"]
                print(f"Claude wants to get an article for {search_term}")
                try:
                    wiki_result = get_article(search_term)
                    tool_response = {
                        "role": "user",
                        "content": [
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_block.id,
                                "content": wiki_result
                            }
                        ]
                    }
                    messages.append({"role": "assistant", "content": response.content})
                    messages.append(tool_response)
                except Exception as e:
                    print(f"Error getting article for {search_term}: {e}")

        elif response.stop_reason == "end_turn":

            # Check if the response contains the final answer tag
            final_answer_block = next((block for block in response.content if block.type == "text" and "<answer>" in block.text), None)

            if final_answer_block:
                # Extract the text within the <answer> tags
                import re
                match = re.search(r"<answer>(.*?)</answer>", final_answer_block.text, re.DOTALL)
                if match:
                    final_answer = match.group(1).strip()
                    display(Markdown(f"**Claude's final answer**:\n\n {final_answer}"))
                else:
                    display(Markdown(f"**Claude's response**:\n\n {final_answer_block.text}"))
                break # Exit loop after finding the final answer
            else:
                print("Claude did not call our tool or provide a final answer tag.")
                display(Markdown(response.content[0].text))
                break # Exit loop if no tool call or final answer tag

        else:
            print(f"Claude stopped with reason: {response.stop_reason}")
            display(Markdown(response.content[0].text))
            break # Exit loop for other stop reasons

In [ ]:
complex_answer_question("Does Sweden, Uganda, or Brazil have more people?")

Claude wants to get an article for Sweden population
Claude wants to get an article for Uganda population
Claude wants to get an article for Brazil population
Claude wants to get an article for Population of Sweden Uganda Brazil


**Claude's final answer**:

 Brazil has the largest population among these three countries, with approximately 214 million people, compared to Uganda's 49.3 million and Sweden's 10.6 million.

# [Lesson 4](https://github.com/anthropics/courses/blob/master/tool_use/05_tool_choice.ipynb)

Tool choice:
- auto – let Claude decide whether to use tools or not (default)
  - Note that Claude is often over-eager to use tools, so take your time writing a detailed system prompt on when to use them and not use them.
- any – let Claude decide which tool to use, but has to be one
- tool – force Claude to use a specific tool

In [ ]:
def web_search(topic):
  print(f"Mock web search for {topic}")

tools = [
    {
        "name": "web_search",
        "description": "A tool to retrieve up to date information on a given topic by searching the web",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search the web for"
                }
            },
            "required": ["topic"] # This was moved inside input_schema
        }
    },
]

from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": user_query}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    response = claude_client.messages.create(
        system=system_prompt,
        model=model,
        messages=messages,
        max_tokens=1000,
        tool_choice={"type": "auto"},
        # Can be auto, any, or pass in a specific tool name like this: tool_choice={"type": "tool", "name": "print_sentiment_scores"}
        tools=tools
    )
    last_content_block = response.content[-1]
    if last_content_block.type == "text":
        print("Claude did NOT call a tool")
        print(f"Assistant: {last_content_block.text}")
    elif last_content_block.type == "tool_use":
        print("Claude wants to use a tool")
        print(last_content_block)

In [ ]:
chat_with_web_search("Who won the 2025 Wimbolden tennis final?")

Claude wants to use a tool
ToolUseBlock(id='toolu_018tEoAXzpML38c6KhgZNvpg', input={'topic': "2025 Wimbledon men's and women's singles champions"}, name='web_search', type='tool_use')


# [Lesson 6](https://github.com/anthropics/courses/blob/master/tool_use/06_chatbot_with_multiple_tools.ipynb)

Building a multi-tool customer support chatbot

In [ ]:
# Setup the fake database

class FakeDatabase:
    def __init__(self):
        self.customers = [
            {"id": "1213210", "name": "John Doe", "email": "john@gmail.com", "phone": "123-456-7890", "username": "johndoe"},
            {"id": "2837622", "name": "Priya Patel", "email": "priya@candy.com", "phone": "987-654-3210", "username": "priya123"},
            {"id": "3924156", "name": "Liam Nguyen", "email": "lnguyen@yahoo.com", "phone": "555-123-4567", "username": "liamn"},
            {"id": "4782901", "name": "Aaliyah Davis", "email": "aaliyahd@hotmail.com", "phone": "111-222-3333", "username": "adavis"},
            {"id": "5190753", "name": "Hiroshi Nakamura", "email": "hiroshi@gmail.com", "phone": "444-555-6666", "username": "hiroshin"},
            {"id": "6824095", "name": "Fatima Ahmed", "email": "fatimaa@outlook.com", "phone": "777-888-9999", "username": "fatimaahmed"},
            {"id": "7135680", "name": "Alejandro Rodriguez", "email": "arodriguez@protonmail.com", "phone": "222-333-4444", "username": "alexr"},
            {"id": "8259147", "name": "Megan Anderson", "email": "megana@gmail.com", "phone": "666-777-8888", "username": "manderson"},
            {"id": "9603481", "name": "Kwame Osei", "email": "kwameo@yahoo.com", "phone": "999-000-1111", "username": "kwameo"},
            {"id": "1057426", "name": "Mei Lin", "email": "meilin@gmail.com", "phone": "333-444-5555", "username": "mlin"}
        ]

        self.orders = [
            {"id": "24601", "customer_id": "1213210", "product": "Wireless Headphones", "quantity": 1, "price": 79.99, "status": "Shipped"},
            {"id": "13579", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 2, "price": 19.99, "status": "Processing"},
            {"id": "97531", "customer_id": "2837622", "product": "Bluetooth Speaker", "quantity": 1, "price": "49.99", "status": "Shipped"},
            {"id": "86420", "customer_id": "3924156", "product": "Fitness Tracker", "quantity": 1, "price": 129.99, "status": "Delivered"},
            {"id": "54321", "customer_id": "4782901", "product": "Laptop Sleeve", "quantity": 3, "price": 24.99, "status": "Shipped"},
            {"id": "19283", "customer_id": "5190753", "product": "Wireless Mouse", "quantity": 1, "price": 34.99, "status": "Processing"},
            {"id": "74651", "customer_id": "6824095", "product": "Gaming Keyboard", "quantity": 1, "price": 89.99, "status": "Delivered"},
            {"id": "30298", "customer_id": "7135680", "product": "Portable Charger", "quantity": 2, "price": 29.99, "status": "Shipped"},
            {"id": "47652", "customer_id": "8259147", "product": "Smartwatch", "quantity": 1, "price": 199.99, "status": "Processing"},
            {"id": "61984", "customer_id": "9603481", "product": "Noise-Cancelling Headphones", "quantity": 1, "price": 149.99, "status": "Shipped"},
            {"id": "58243", "customer_id": "1057426", "product": "Wireless Earbuds", "quantity": 2, "price": 99.99, "status": "Delivered"},
            {"id": "90357", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 1, "price": 19.99, "status": "Shipped"},
            {"id": "28164", "customer_id": "2837622", "product": "Wireless Headphones", "quantity": 2, "price": 79.99, "status": "Processing"}
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")

        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None

    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"

db = FakeDatabase()

In [ ]:
# Test that the database works

print(f"Fetch user:\n{db.get_user('username', 'adavis')}\n")
print(f"Fetch customer orders:\n{db.get_customer_orders('1213210')}\n")
print(f"Fetch single order:\n{db.get_order_by_id('24601')}\n")

Fetch user:
{'id': '4782901', 'name': 'Aaliyah Davis', 'email': 'aaliyahd@hotmail.com', 'phone': '111-222-3333', 'username': 'adavis'}

Fetch customer orders:
[{'id': '24601', 'customer_id': '1213210', 'product': 'Wireless Headphones', 'quantity': 1, 'price': 79.99, 'status': 'Shipped'}, {'id': '13579', 'customer_id': '1213210', 'product': 'Smartphone Case', 'quantity': 2, 'price': 19.99, 'status': 'Processing'}, {'id': '90357', 'customer_id': '1213210', 'product': 'Smartphone Case', 'quantity': 1, 'price': 19.99, 'status': 'Shipped'}]

Fetch single order:
{'id': '24601', 'customer_id': '1213210', 'product': 'Wireless Headphones', 'quantity': 1, 'price': 79.99, 'status': 'Shipped'}



In [ ]:
# Setup Claude tools

tools = [
    {
        "name": "get_user",
        "description": "Looks up a user by email, phone, or username.",
        "input_schema": {
            "type": "object",
            "properties": {
                "key": {
                    "type": "string",
                    "enum": ["email", "phone", "username"],
                    "description": "The attribute to search for a user by (email, phone, or username)."
                },
                "value": {
                    "type": "string",
                    "description": "The value to match for the specified attribute."
                }
            },
            "required": ["key", "value"]
        }
    },
    {
        "name": "get_order_by_id",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "get_customer_orders",
        "description": "Retrieves the list of orders belonging to a user based on a user's customer id.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The customer_id belonging to the user"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on a provided order_id.  Only orders that are 'processing' can be cancelled",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order_id pertaining to a particular order"
                }
            },
            "required": ["order_id"]
        }
    }
]

In [ ]:
# Setup a function to handle each type of tool call

def process_tool_call(tool_name, tool_input):
    if tool_name == "get_user":
        return db.get_user(tool_input["key"], tool_input["value"])
    elif tool_name == "get_order_by_id":
        return db.get_order_by_id(tool_input["order_id"])
    elif tool_name == "get_customer_orders":
        return db.get_customer_orders(tool_input["customer_id"])
    elif tool_name == "cancel_order":
        return db.cancel_order(tool_input["order_id"])

In [ ]:
# Setup looping chatbot

def customer_support_chat():
    user_message = input("\nUser:")
    messages = [{"role": "user", "content": user_message}]
    system_prompt = """
    You are a customer support chatbot for an online retailer.
    Your job is to help users look up their account, orders, and cancel orders.
    Be helpful and brief in your responses.

    You have access to a set of tools, but only use them when needed.
    If you do not have enough information to use a tool correctly, ask the user follow-up questions.

    When the user’s issue is resolved and no further help is needed, you may end the conversation by replying with a short goodbye message like "You're welcome. Have a great day.", then print the tag <conversation_end>.
    """

    while True:

        # If the last message is from the assistant, get another input from the user
        if messages[-1].get("role") == "assistant":
            user_message = input("\nUser:")
            messages.append({"role": "user", "content": user_message})

        # Send a request to Claude
        response = claude_client.messages.create(
            model=model,
            max_tokens=2000,
            tools=tools,
            messages=messages,
            system=system_prompt
        )

        # Update messages to include Claude's response
        messages.append(
            {"role": "assistant", "content": response.content}
        )

        # If Claude stops because it wants to use a tool:
        if response.stop_reason == "tool_use":
            tool_block = next((block for block in response.content if block.type == "tool_use"), None)

            tool_name = tool_block.name
            tool_input = tool_block.input

            print(f"–––– Claude wants to use the {tool_name} tool ––––")

            # Run the function to use the tool
            tool_result = process_tool_call(tool_name, tool_input)

            # Add the tool result to our messages
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_block.id,
                            "content": str(tool_result)
                        }
                    ]
                }
            )
        else:
            # If Claude chooses not to use a tool, print out text
            print(f"\nCustomer support says: {response.content[0].text}")

            # If Claude decides to end the conversation
            if "<conversation_end>" in response.content[0].text:
                break

In [ ]:
# "Can you find my order? My username is manderson"

customer_support_chat()


User:Can you find my order? My username is manderson
–––– Claude wants to use the get_user tool ––––
–––– Claude wants to use the get_customer_orders tool ––––

Customer support says: I found one order for you:
- Order ID: 47652
- Product: Smartwatch
- Quantity: 1
- Price: $199.99
- Status: Processing

Is there anything specific you would like to know about this order? I can provide more details or help you cancel it if needed.

User:No that's all I needed

Customer support says: You're welcome. Have a great day!

<conversation_end>


In [ ]:
# Opus version with thinking

def extract_reply(text):
    pattern = r'<reply>(.*?)</reply>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return None

def thinking_support_chatbot():
    user_message = input("\nUser:")
    messages=[{"role": "user", "content": user_message}]

    system_prompt = """
    You are a customer support chat bot for an online retailer.
    Your job is to help users look up their account, orders, and cancel orders.
    Be helpful and brief in your responses.
    You have access to a set of tools, but only use them when needed.
    If you do not have enough information to use a tool correctly, ask a user follow up questions to get the required inputs.
    Do not call any of the tools unless you have the required data from a user.

    In each conversational turn, you will begin by thinking about your response.
    Once you're done, you will write a user-facing response.
    It's important to place all user-facing conversational responses in <reply></reply> XML tags to make them easy to parse.

    When the user’s issue is resolved and no further help is needed, you may end the conversation by replying with a short goodbye message like "You're welcome. Have a great day.", then print the tag <conversation_end>.
    """

    while True:

        # If the last message was from the assistant, ask the user for input
        if messages[-1].get("role") == "assistant":
            user_message = input("\nUser:")
            messages.append({"role": "user", "content": user_message})

        # Send a request to Claude
        response = claude_client.messages.create(
            model="claude-sonnet-4-0",
            max_tokens=2000,
            tools=tools,
            messages=messages,
            system=system_prompt
        )

        # Update messages to include Claude's response
        messages.append(
            {"role": "assistant", "content": response.content}
        )

        # If Claude stops because it wants to use a tool:
        if response.stop_reason == "tool_use":
            tool_block = next((block for block in response.content if block.type == "tool_use"), None)

            tool_name = tool_block.name
            tool_input = tool_block.input

            print(f"–––– Claude wants to use the {tool_name} tool ––––")

            # Run the function to use the tool
            tool_result = process_tool_call(tool_name, tool_input)

            # Add the tool result to our messages
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_block.id,
                            "content": str(tool_result)
                        }
                    ]
                }
            )
        else:
            # If Claude chooses not to use a tool, print out text
            # model_reply = extract_reply(response.content[0].text)
            print(f"\nCustomer support says: {response.content[0].text}")

            # If Claude decides to end the conversation
            if "<conversation_end>" in response.content[0].text:
                break

In [ ]:
thinking_support_chatbot()


User:can u help me cancel an order?

Customer support says: I'd be happy to help you cancel an order! To do this, I'll need your order ID. 

<reply>
I'd be happy to help you cancel an order! Could you please provide me with your order ID?
</reply>

User:i dont know it

Customer support says: I can help you find your order ID. To look up your orders, I'll need to identify your account first. 

<reply>
No problem! I can look up your orders for you. To find your account, could you please provide me with either:
- Your email address
- Your phone number
- Your username

Once I find your account, I'll be able to show you your orders and help you cancel the one you need.
</reply>

User:adavis
–––– Claude wants to use the get_user tool ––––
–––– Claude wants to use the get_customer_orders tool ––––

Customer support says: <reply>
Hi Aaliyah! I found your account and see you have one order:

**Order #54321**
- Product: Laptop Sleeve
- Quantity: 3
- Price: $24.99
- Status: Shipped

Unfortunatel